In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/company-data/data/hr_policy_ar.txt
/kaggle/input/company-data/data/tips_hindawi_data_ar.txt
/kaggle/input/company-data/data/support_tickets_en.csv
/kaggle/input/company-data/data/leave_policy_ar.txt
/kaggle/input/company-data/data/leave_policy_en.pdf
/kaggle/input/company-data/data/internal_faqs_ar.txt
/kaggle/input/company-data/data/tips_hindawi_data_en.txt
/kaggle/input/company-data/data/it_guidelines_en.txt
/kaggle/input/company-data/data/internal_faqs_en.txt
/kaggle/input/company-data/data/support_tickets_ar.csv
/kaggle/input/company-data/data/hr_policy.pdf
/kaggle/input/company-data/data/it_guidelines_ar.txt


In [2]:
!pip install -q fastapi uvicorn pyngrok transformers==4.52.4 accelerate faiss-cpu langchain langchain-community pypdf sentence-transformers langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 99.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 77.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.47.0 which is in

In [3]:
from huggingface_hub import login
login()

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

print("🦙 Loading Meta Llama 3 8B Instruct...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
).eval()

print("Llama 3 loaded successfully!")


def generate_text(prompt: str, max_length: int = None) -> str:
    """Generate text using Llama 3 with chat template"""
    if max_length is None:
        max_length = 1000
    
    messages = [{"role": "user", "content": prompt}]
    
    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature=0.7
        )
    
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    if "assistant" in full_response.lower():
        parts = full_response.split("assistant")
        answer = parts[-1].strip()
    else:
        answer = full_response.strip()
    
    return answer

🦙 Loading Meta Llama 3 8B Instruct...


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Llama 3 loaded successfully!


In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

DATA_DIR = "/kaggle/input/company-data/data" 
documents = []
file_count = 0

for filename in os.listdir(DATA_DIR):
    filepath = os.path.join(DATA_DIR, filename)
    
    try:
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(filepath)
            file_count += 1
        elif filename.endswith('.txt'):
            loader = TextLoader(filepath, encoding='utf-8')
            file_count += 1
        else:
            continue
        
        docs = loader.load()
        documents.extend(docs)
        print(f"   ✓ Loaded: {filename}")
    
    except Exception as e:
        print(f"   ✗ Error loading {filename}: {str(e)}")

print(f"\nLoaded {len(documents)} document(s) from {file_count} file(s)")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", " ", ""]
)
chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
)
vectordb = FAISS.from_documents(chunks, embeddings)
print("✅ Vector store ready!")


def search_documents(query: str, k: int = None):
    """Search for relevant documents"""
    if k is None:
        k = 3
    return vectordb.similarity_search(query, k=k)

   ✓ Loaded: hr_policy_ar.txt
   ✓ Loaded: tips_hindawi_data_ar.txt
   ✓ Loaded: leave_policy_ar.txt
   ✓ Loaded: leave_policy_en.pdf
   ✓ Loaded: internal_faqs_ar.txt
   ✓ Loaded: tips_hindawi_data_en.txt
   ✓ Loaded: it_guidelines_en.txt
   ✓ Loaded: internal_faqs_en.txt
   ✓ Loaded: hr_policy.pdf
   ✓ Loaded: it_guidelines_ar.txt

Loaded 10 document(s) from 10 file(s)
Created 28 chunks


/tmp/ipykernel_55/2207542100.py:40: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Vector store ready!


In [7]:
from langdetect import detect

def detect_language(text: str) -> str:
    """Detect if text is Arabic or English"""
    try:
        lang = detect(text)
        return lang
    except:
        return "en"


def create_rag_prompt(query: str, context: str) -> str:
    """Create RAG prompt optimized for Llama 3 with multilingual support"""
    
    # Detect language
    lang = detect_language(query)
    
    if lang == "ar":
        # Arabic prompt
        return f"""أنت مساعد ذكاء اصطناعي مفيد للشركات. مهمتك الإجابة على أسئلة الموظفين باستخدام المعلومات من وثائق الشركة المقدمة أدناه فقط.

تعليمات مهمة:
- أجب فقط باستخدام المعلومات من السياق أدناه
- إذا لم تكن الإجابة في السياق، قل "ليس لدي معلومات كافية للإجابة على هذا السؤال بناءً على المستندات المتاحة"
- كن موجزاً وواضحاً ومهنياً
- لا تختلق أو تفترض معلومات

السياق من وثائق الشركة:
{context}

سؤال الموظف: {query}

الإجابة:"""
    else:
        # English prompt
        return f"""You are a helpful enterprise AI assistant. Your job is to answer employee questions using only the information from company documents provided below.

IMPORTANT INSTRUCTIONS:
- Answer ONLY using information from the context below
- If the answer is not in the context, say "I don't have enough information to answer this based on the available documents"
- Be concise, clear, and professional
- Do not make up or assume information
- CRITICAL: Respond in the SAME language as the question

CONTEXT FROM COMPANY DOCUMENTS:
{context}

EMPLOYEE QUESTION: {query}

ANSWER:"""


def ask_question(question: str, max_length: int = None) -> dict:
    """Main RAG pipeline with Llama 3 - Now supports Arabic and English"""
    
    # Step 1: Retrieve relevant documents
    docs = search_documents(question)
    
    # Step 2: Combine context
    context = "\n\n".join([
        f"[Document {i+1}]\n{doc.page_content}" 
        for i, doc in enumerate(docs)
    ])
    
    # Step 3: Create prompt (automatically detects language)
    prompt = create_rag_prompt(question, context)
    
    # Step 4: Generate answer
    answer = generate_text(prompt, max_length)
    
    # Step 5: Clean answer
    if "ANSWER:" in answer or "الإجابة:" in answer:
        if "ANSWER:" in answer:
            answer = answer.split("ANSWER:")[-1].strip()
        elif "الإجابة:" in answer:
            answer = answer.split("الإجابة:")[-1].strip()
    
    return {
        "question": question,
        "answer": answer,
        "language": detect_language(question)
    }

print("✅ RAG Pipeline ready with Arabic and English support!")

✅ RAG Pipeline ready with Arabic and English support!


In [8]:
from fastapi import FastAPI, HTTPException, Header
from pydantic import BaseModel
from typing import Optional

API_KEY = "secret226"
NGROK_TOKEN = "3825DpWFqNVWaCztnWjTVZAZDw1_3LoS6zTvipBMY6HYoCV8k"

app = FastAPI(
    title="🦙 Enterprise AI Assistant - Llama 3",
    description="RAG-based Q&A system powered by Meta Llama 3 8B Instruct with Arabic & English support",
    version="2.0.0"
)


class QueryRequest(BaseModel):
    question: str
    max_length: Optional[int] = None


@app.get("/")
async def root():
    return {
        "message": "🦙 Enterprise AI Knowledge Assistant API",
        "model": "Meta-Llama-3-8B-Instruct",
        "status": "running",
        "version": "2.0.0",
        "languages": ["English", "Arabic"],
        "endpoints": {
            "ask": "/ask - RAG-based Q&A",
            "generate": "/generate - Direct generation",
            "health": "/health - Health check"
        }
    }


@app.get("/health")
async def health():
    return {
        "status": "healthy",
        "model": "Meta-Llama-3-8B-Instruct",
        "languages_supported": ["en", "ar"]
    }


@app.post("/ask")
async def api_ask(request: QueryRequest, authorization: str = Header(...)):
    """Ask a question with RAG - Supports Arabic and English"""
    # Verify API key
    if not authorization.startswith("Bearer "):
        raise HTTPException(status_code=401, detail="Invalid authorization format")
    
    token = authorization.replace("Bearer ", "")
    if token != API_KEY:
        raise HTTPException(status_code=401, detail="Invalid API key")
    
    # Process question
    try:
        result = ask_question(request.question, request.max_length)
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.post("/generate")
async def api_generate(request: QueryRequest, authorization: str = Header(...)):
    """Direct text generation - Supports Arabic and English"""
    # Verify API key
    if not authorization.startswith("Bearer "):
        raise HTTPException(status_code=401, detail="Invalid authorization format")
    
    token = authorization.replace("Bearer ", "")
    if token != API_KEY:
        raise HTTPException(status_code=401, detail="Invalid API key")
    
    try:
        response = generate_text(request.question, request.max_length)
        return {"response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


print("✅ FastAPI app configured for Llama 3 with multilingual support")

✅ FastAPI app configured for Llama 3 with multilingual support


In [ ]:
import uvicorn
import threading
import socket
from pyngrok import ngrok, conf
import time


def get_free_port():
    """Find available port"""
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port


port = get_free_port()

conf.get_default().auth_token = NGROK_TOKEN

public_url = ngrok.connect(port).public_url
print(f"\n🌐 Public URL: {public_url}")


def run_server():
    uvicorn.run(
        app,
        host="0.0.0.0",
        port=port,
        log_level="info",
        access_log=False
    )


server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

print("\n✅ Server is running with Arabic support!")

# Keep the notebook alive
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n⏹️ Server stopped")

                                                                                                    
🌐 Public URL: https://thigmotropic-geraldine-nonelliptically.ngrok-free.dev

✅ Server is running with Arabic support!


INFO:     Started server process [55]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:49077 (Press CTRL+C to quit)
